# Notebook: Filter Sentences from Reviews to create Dataset for Annotation

## Packages

In [1]:
from bs4 import BeautifulSoup
from langdetect import detect
import pandas as pd
import json
import nltk
import uuid
from nltk.tokenize import sent_tokenize
import re

## Settings

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nils_hellwig/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Constants

In [3]:
REVIEWS_PATH = "reviews_dataset/reviews.csv"
RANDOM_STATE = 43

## Code

### Load Dataset

In [4]:
df_reviews = pd.read_csv(REVIEWS_PATH)

In [5]:
df_reviews

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization
0,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Die LOC schmeckte leicht säuerlich, der kleine...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...
1,1,907168709,2005104,0,Wir sind hier nicht in Bayern!,2023-08-01,173stefanh,"München, Deutschland","Wir sind hier nicht in LOC, sind die freundlic...",1.0,berlin,Augustiner am Gendarmenmarkt,de,"Wir sind hier nicht in Bayern, sind die freund..."
2,2,902540630,2005104,0,Heute leider zum abgewöhnen,2023-07-12,Patrick L,"Rüdersdorf, Deutschland",Ich freute mich sehr auf das Wienerschnitzel u...,3.0,berlin,Augustiner am Gendarmenmarkt,de,Ich freute mich sehr auf das Wienerschnitzel u...
3,3,895783388,2005104,0,Teils-Teils,2023-06-18,J9324NNpeterr,"Stellenbosch, Südafrika",Meine Erfahrungen in diesem Jahr waren gemisch...,4.0,berlin,Augustiner am Gendarmenmarkt,de,Meine Erfahrungen in diesem Jahr waren gemisch...
4,4,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",ich hätte gerne etwas zum Essen gesagt. Aber e...,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3596,3596,889964622,5207419,0,Nicht besonderes,2023-05-11,Mobile28959210843,"Mönchengladbach, Deutschland",Das Essen ist okay mehr aber auch nicht. Ambie...,3.0,mönchengladbach,Brauerei zum Stefanus,de,Das Essen ist okay mehr aber auch nicht. Ambie...
3597,3597,870345760,5207419,0,"Ungebührender Empfang, sehr gutes Essen",2022-12-01,Korich,"Görlitz, Deutschland",Zunächst einmal wurden wir von einem Mann mit ...,4.0,mönchengladbach,Brauerei zum Stefanus,de,Zunächst einmal wurden wir von einem Mann mit ...
3598,3598,865168700,5207419,0,Ich liebe diese Haxe,2022-10-19,BibiH41366,NaN,Wir sind nicht zum ersten Mal aber auch defini...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Wir sind nicht zum ersten Mal aber auch defini...
3599,3599,863841135,5207419,0,Gutes Essen mit Biergartenambiente,2022-10-09,788heinzn,"Köln, Deutschland","Gutes Essen, Portionen sehr groß (für uns zu g...",4.0,mönchengladbach,Brauerei zum Stefanus,de,"Gutes Essen, Portionen sehr groß (für uns zu g..."


### Filter Sentences

In [6]:
df_reviews_sentences = pd.DataFrame(columns=list(df_reviews.columns) + ['sentence_idx', "id"])

for idx, row in df_reviews.iterrows():
    sentences = sent_tokenize(row['text'], language="german")
    sentence_index = 0
    for sentence in sentences:
        if len(sentence) > 1:
            new_row = row.copy() 
            new_row['text'] = sentence 
            new_row['id'] = str(uuid.uuid4())
            new_row['sentence_idx'] = sentence_index
            df_reviews_sentences.loc[len(df_reviews_sentences)] = new_row
            sentence_index += 1

In [7]:
df_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id
0,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Die LOC schmeckte leicht säuerlich, der kleine...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,0,fd2b64dc-36dc-4f75-8e67-529927aff86b
1,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Der Schweinebraten war lauwarm und der Kaisers...,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,1,6385959a-5c90-489e-9fa3-1a1336bd5600
2,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Ich bin so enttäuschst.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,2,56ae6978-ceb8-4288-b2a0-eee0083e7ef7
3,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland",Gehe ich nicht mehr hin.,2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,3,a25f63b7-d0ee-498e-adb9-b6cebf54a579
4,0,908201159,2005104,0,Qualität des Essens hat sehr stark nachgelassen.,2023-08-06,JK44892,"Bochum, Deutschland","Schlecht kann ich auch woanders, besonders bei...",2.0,berlin,Augustiner am Gendarmenmarkt,de,Die Leberknödelsuppe schmeckte leicht säuerlic...,4,01987407-97b3-4176-88b0-4fad16b27678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15344,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,Das Essen schmeckt aber generell grandios.,5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,1,7ee8c383-561a-4af3-990f-f7e90c860ac5
15345,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,Der Beilagensalat ist auch der Kracher: frisch...,5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,2,cd0fd857-ec95-44bd-bad7-024a88ec0d0b
15346,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,"Im Stefanus gibt es auch selbstgebrautes, sehr...",5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,3,90ed7b9c-da9f-4ba4-a641-3de2b383f8ce
15347,3600,861397747,5207419,0,Das Schnitzelparadies!,2022-09-22,vereny2020,NaN,"Der Service ist sehr nett, zuvorkommend und he...",5.0,mönchengladbach,Brauerei zum Stefanus,de,Ich bin im Schnitzelparadies <3 Die Schnitzel ...,4,02186876-8469-4d20-be67-8688ca6604c1


### Filter Non-German Sentences

Even if the language code = "de", we have observed that some sentences are in other languagesthan german. These are excluded as well with the help of Google's language-detection.

In [8]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'Unknown'

df_reviews_sentences['detected_language'] = df_reviews_sentences['text'].apply(detect_language)
df_reviews_sentences[df_reviews_sentences['detected_language'] != 'de']

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
17,4,890917149,2005104,0,schlecht verwaltetes Chaos,2023-05-18,Harald F,"Berlin, Deutschland",Es war viel los.,1.0,berlin,Augustiner am Gendarmenmarkt,de,ich hätte gerne etwas zum Essen gesagt. Aber e...,3,f6274801-89c5-4db0-b757-d30950e62141,es
66,13,866452621,2005104,1,Mind this touri-gap,2022-10-29,452stephand,"Nürnberg, Deutschland",Fazit: mind this touri-gap,1.0,berlin,Augustiner am Gendarmenmarkt,de,Wunderschöne Location mit mangelhafter Leitung...,1,e7cbc719-3082-4785-a7e1-cc2c96e6c82f,en
93,17,856900319,2005104,1,Deftig bayrisch,2022-08-28,tatrab,"Kaiserslautern, Deutschland",Kann man mal hin,3.0,berlin,Augustiner am Gendarmenmarkt,de,Vorzubereitete Speisen in bayrischer Manier zu...,2,c11f0313-f748-449f-85d9-1356b7938519,tl
113,22,847064431,2005104,2,Enttäuschend,2022-07-09,HerSchnu,"Erfurt, Deutschland",Fing schon gut an….,2.0,berlin,Augustiner am Gendarmenmarkt,de,In jedem bayerischem Gasthof isst man besser… ...,2,899868fc-cba3-41f0-b59a-fdda32238f46,en
131,25,883653345,947082,0,Tolle Burger!,2023-03-26,dittelbrunn,"Lübeck, Deutschland",Ketchup.,5.0,berlin,Burgermeister,de,Wir waren in der Filiale am S-Bahnhof Eberswal...,2,aed70c94-560e-4a59-8d88-ff0412797388,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15231,3570,862649725,13144674,6,Einfach eine Freude und ein Genuss,2022-09-30,Bernd W,NaN,Klasse.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,"Okay, ich koche gern selbst. Wenn ich aber mal...",4,d4210b2c-9bd5-4267-bf9d-1a4be7ce5351,et
15241,3570,862649725,13144674,6,Einfach eine Freude und ein Genuss,2022-09-30,Bernd W,NaN,Place to be.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,"Okay, ich koche gern selbst. Wenn ich aber mal...",14,782850d1-1649-426b-9054-48fd79fb1c95,en
15249,3572,861638375,13144674,6,"Handwerklich einwandfrei, Hohe Qualität und ei...",2022-09-24,Tilmann S,NaN,Top.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Eher zufällig kehrten wir als Ortsunkundige im...,4,06b243b5-1eaf-49f8-b7b3-f33ec0eccd74,en
15280,3583,847136487,13144674,7,Super Essen Super Service,2022-07-10,U4608RGalexs,"Schwalbach, Deutschland",Alles was wir essen war super.,5.0,mönchengladbach,Frenzen Restaurant & Event,de,Hervorragend wie immer. Immer wenn wir aus Fra...,2,eb8adac7-7660-461b-97fd-f31836a5a238,en


In [9]:
df_reviews_sentences = df_reviews_sentences[df_reviews_sentences['detected_language'] == 'de']

### Balancing

In [10]:
df_balanced_reviews_sentences = df_reviews_sentences.groupby(['rating'], group_keys=False).apply(lambda x: x.sample(800, replace=False, random_state=RANDOM_STATE))
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Randomisierung

In [11]:
df_balanced_reviews_sentences = df_balanced_reviews_sentences.sample(frac=1, random_state=RANDOM_STATE)  # 'frac=1' mischt den gesamten DataFrame
df_balanced_reviews_sentences.reset_index(drop=True, inplace=True)

### Check If Sentences are Unique

In [12]:
df_balanced_reviews_sentences.groupby(["review_id", "sentence_idx"]).size().reset_index(name="count")

,review_id,sentence_idx,count
0,845636774,0,1
1,845636774,1,1
2,845636774,2,1
3,845636774,3,1
4,845658907,0,1
...,...,...,...
3995,916494891,3,1
3996,916494891,4,1
3997,916494891,7,1
3998,916548709,0,1


### Alternate examples based on rating

In [13]:
df_balanced_reviews_sentences["rating"].value_counts()

rating
1.0    800
3.0    800
5.0    800
2.0    800
4.0    800
Name: count, dtype: int64

In [14]:
dfs = [df_balanced_reviews_sentences[df_balanced_reviews_sentences["rating"] == i] for i in range(1, 6)]
df_alternating = pd.DataFrame(columns=df_balanced_reviews_sentences.columns)

max_rows = max(len(df) for df in dfs)
for i in range(max_rows):
    for k in range(5):
        if i < len(dfs[k]):
            df_alternating = pd.concat([df_alternating, dfs[k].iloc[[i]]])

df_balanced_reviews_sentences = df_alternating.reset_index(drop=True)

In [15]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,3124,885663660,908561,0,Schon lange nicht mehr so schlecht gegessen,2023-04-10,michaellL5805JV,"Lissabon, Portugal","Suppe war nicht heiß, kümmerte den Service abe...",1.0,karlsruhe,Anders auf dem Turmberg,de,Schon lange nicht mehr so schlecht gegessen! A...,3,29201856-9893-4eec-a44a-82174cbd867c,de
1,338,909600304,742684,0,Keine Bedienung,2023-08-11,Patrick F,NaN,Keine BedienungSind 1 1/2 Stunden im Gastgarte...,2.0,münchen,Hofbräuhaus,de,Keine BedienungSind 1 1/2 Stunden im Gastgarte...,0,5ed24454-faec-4944-b968-b84b9fa5cb6e,de
2,988,884767828,965262,0,Ein Besuch in einem Düsseldorfer Brauhaus!,2023-04-04,Andreas K,"Leipzig, Deutschland",Im Vergleich zum Personal im Schlüssel und LOC...,3.0,düsseldorf,Uerige Hausbrauerei,de,Wir sind an einem Sonntagnachmittag durch mehr...,6,294fe7a8-c829-4706-89f7-cf88341c4828,de
3,1552,867801811,718273,0,gutes Essen in der Innenstadt,2022-11-09,Zwerg74,Aachen,Wir würden wiederkommen.,4.0,dortmund,Pfefferkorn,de,Das Essen war geschmacklich toll und Preis-Lei...,1,81343575-4711-4a44-bcb1-932dc3c2e0c0,de
4,3396,860042926,2049344,1,"Traditionscafé, das auch in Wie sein könnte, e...",2022-09-14,gmsorg,"Bad Homburg, Deutschland","LOC, eine beeindruckende Kuchen- und Tortenaus...",5.0,wiesbaden,Cafe Maldaner,de,Mitten in Wiesbaden in der belebten Fußgängerz...,1,b654d8c6-c609-48d6-9bbb-cac33caeff87,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3503,855090309,6541566,1,Leider nicht zu empfehlen,2022-08-19,Michael P,"Erkelenz, Deutschland","Ein weiteres absolutes NoGo war, dass sich wäh...",1.0,mönchengladbach,L'Osteria Mönchengladbach,de,Leider waren wir sehr unzufrieden und besuchen...,8,0764555d-88c3-4816-9177-c6ab11c87f9d,de
3996,360,897549645,947503,0,Bei der lausigen Küche hilft die prächtige Dek...,2023-06-24,Happiness24765324521,"Paris, Frankreich","Sehr schade, da sitzt man im ""LOC"", einem der ...",2.0,münchen,Augustiner Stammhaus,de,"Sehr schade, da sitzt man im ""Arkadengarten"", ...",0,624cc1d8-2334-4e13-9a6b-c9767c68baf3,de
3997,380,866848042,947503,2,Kurzer Imbiss,2022-11-01,509markuss,"Küsnacht, Schweiz",Gestresste überhebliche Kellner und das Essen ...,3.0,münchen,Augustiner Stammhaus,de,"Wir kannten das Restaurant von früher, aber al...",1,43e534c1-b1f6-4692-a9b6-71b398d431cf,de
3998,3354,898452974,2640161,0,ein gelungener Abend,2023-06-27,garyaugs,"Augsburg, Deutschland",Im Rahmen einer Führung sind wir zu Besuch im ...,4.0,augsburg,Riegele Wirtshaus,de,Im Rahmen einer Führung sind wir zu Besuch im ...,0,fd93a399-1e35-466a-a8d7-54c6fd77b6bf,de


### Store as .csv 

In [16]:
df_reviews_sentences.to_csv("reviews_dataset/reviews_sentences.csv")

In [17]:
df_balanced_reviews_sentences.to_csv("reviews_dataset/balanced_reviews_sentences.csv")

In [18]:
df_balanced_reviews_sentences

,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,3124,885663660,908561,0,Schon lange nicht mehr so schlecht gegessen,2023-04-10,michaellL5805JV,"Lissabon, Portugal","Suppe war nicht heiß, kümmerte den Service abe...",1.0,karlsruhe,Anders auf dem Turmberg,de,Schon lange nicht mehr so schlecht gegessen! A...,3,29201856-9893-4eec-a44a-82174cbd867c,de
1,338,909600304,742684,0,Keine Bedienung,2023-08-11,Patrick F,NaN,Keine BedienungSind 1 1/2 Stunden im Gastgarte...,2.0,münchen,Hofbräuhaus,de,Keine BedienungSind 1 1/2 Stunden im Gastgarte...,0,5ed24454-faec-4944-b968-b84b9fa5cb6e,de
2,988,884767828,965262,0,Ein Besuch in einem Düsseldorfer Brauhaus!,2023-04-04,Andreas K,"Leipzig, Deutschland",Im Vergleich zum Personal im Schlüssel und LOC...,3.0,düsseldorf,Uerige Hausbrauerei,de,Wir sind an einem Sonntagnachmittag durch mehr...,6,294fe7a8-c829-4706-89f7-cf88341c4828,de
3,1552,867801811,718273,0,gutes Essen in der Innenstadt,2022-11-09,Zwerg74,Aachen,Wir würden wiederkommen.,4.0,dortmund,Pfefferkorn,de,Das Essen war geschmacklich toll und Preis-Lei...,1,81343575-4711-4a44-bcb1-932dc3c2e0c0,de
4,3396,860042926,2049344,1,"Traditionscafé, das auch in Wie sein könnte, e...",2022-09-14,gmsorg,"Bad Homburg, Deutschland","LOC, eine beeindruckende Kuchen- und Tortenaus...",5.0,wiesbaden,Cafe Maldaner,de,Mitten in Wiesbaden in der belebten Fußgängerz...,1,b654d8c6-c609-48d6-9bbb-cac33caeff87,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3503,855090309,6541566,1,Leider nicht zu empfehlen,2022-08-19,Michael P,"Erkelenz, Deutschland","Ein weiteres absolutes NoGo war, dass sich wäh...",1.0,mönchengladbach,L'Osteria Mönchengladbach,de,Leider waren wir sehr unzufrieden und besuchen...,8,0764555d-88c3-4816-9177-c6ab11c87f9d,de
3996,360,897549645,947503,0,Bei der lausigen Küche hilft die prächtige Dek...,2023-06-24,Happiness24765324521,"Paris, Frankreich","Sehr schade, da sitzt man im ""LOC"", einem der ...",2.0,münchen,Augustiner Stammhaus,de,"Sehr schade, da sitzt man im ""Arkadengarten"", ...",0,624cc1d8-2334-4e13-9a6b-c9767c68baf3,de
3997,380,866848042,947503,2,Kurzer Imbiss,2022-11-01,509markuss,"Küsnacht, Schweiz",Gestresste überhebliche Kellner und das Essen ...,3.0,münchen,Augustiner Stammhaus,de,"Wir kannten das Restaurant von früher, aber al...",1,43e534c1-b1f6-4692-a9b6-71b398d431cf,de
3998,3354,898452974,2640161,0,ein gelungener Abend,2023-06-27,garyaugs,"Augsburg, Deutschland",Im Rahmen einer Führung sind wir zu Besuch im ...,4.0,augsburg,Riegele Wirtshaus,de,Im Rahmen einer Führung sind wir zu Besuch im ...,0,fd93a399-1e35-466a-a8d7-54c6fd77b6bf,de


In [19]:
df_balanced_reviews_sentences["rating"].value_counts()

rating
1.0    800
2.0    800
3.0    800
4.0    800
5.0    800
Name: count, dtype: int64